In [1]:
import obi_one as obi

from pathlib import Path
import bluepysnap as snap

circ_path = Path("/Users/mwr/Documents/artefacts/SONATA/O1_data/circuit_config.json")
node_set = "S1nonbarrel_neurons"
node_id = 1337

circ = snap.Circuit(circ_path)


# Synapse selections

We demonstrate different ways of selecting subsets of afferent synapses on a neuron in a circuit.

The output of the selectors is always a pandas.DataFrame with one row per selected synapse. 

The DataFrame contains sufficient information in its columns to identify the synapses unequivocally. It specifies both the name of their edge population and the edge id within the population; this tuple is the identifier of an edge in Sonata. Additionally, the node population and node id of the presynaptic neuron are returned. "afferent_section_id", etc. are provided to specify the location of the synapse on the morphology, for example for visualizations.

Finally, some other, potentially useful statistics are also returned.

## Case 1: Random, unbiased selection

However, the second & third example show that section type, synapse class and node population can be filtered against.

Also note that both intrinsic and extrinsic synapses are considered! To avoid that, set "pre_node_population" to a list of non-virtual populations.

In [2]:
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3
)
display(tst.synapses_on(circ, node_set, node_id))

tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    section_types=(1,),
    pre_synapse_class="INH",
    pre_node_populations=("S1nonbarrel_neurons",)
)
display(tst.synapses_on(circ, node_set, node_id))

tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    section_types=(3, 4),
    pre_node_populations=("POm",)
)
display(tst.synapses_on(circ, node_set, node_id))


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
868,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186554,124,24,0.588084,113999,1337,3
477,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186163,142,6,0.880533,26229,1337,3
215,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185901,147,7,0.953837,14544,1337,3


/Users/mwr/Documents/repositories/obi-one/obi_one/scientific/afferent_synapse_finder/find_specified_afferent_synapses.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pre_node_props = pandas.concat(pre_node_props, axis=0, names=["source_population", "edge_population"],


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type
1619,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1187305,0,0,0.0,173904,1337,INH,1
1277,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186963,0,0,0.0,141883,1337,INH,1
1276,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186962,0,0,0.0,141883,1337,INH,1


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
1820,POm,POm__S1nonbarrel_neurons__chemical,102540,166,68,1.033777,4794,1337,4
1831,POm,POm__S1nonbarrel_neurons__chemical,102551,162,91,0.347443,6022,1337,4
1819,POm,POm__S1nonbarrel_neurons__chemical,102539,168,21,3.185710,3832,1337,4


### Note on insufficient synapses

In general, if there are fewer valid synapses than requested, no exception is raised. Instead simply the lower number of synapses is returned.
The code has a "raise_insufficient" flag that can change the behavior, but it currently is not exposed to the user.

Another question is: What happens if filter, e.g., by presynaptic synapse class, but the presynaptic nodes do not have a synapse_class attribute? This is set by the optional consider_nan_pass parameter.

In [3]:
# Requesting EXC synapses on the axon. There are none. This will return empty
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    section_types=(2, ),
    pre_synapse_class="EXC"
)
display(tst.synapses_on(circ, node_set, node_id))

# Default behavior: Even though the POm population has no synapse_class attribute, they will not be filtered. This returns 3 synapses
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    pre_synapse_class="EXC",
    pre_node_populations=("POm",)
)
display(tst.synapses_on(circ, node_set, node_id))

# More stric: POm, which have no synapse_class will be filtered. Hence, no synapses can be returned.
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    pre_synapse_class="EXC",
    pre_node_populations=("POm",),
    consider_nan_pass=False
)
display(tst.synapses_on(circ, node_set, node_id))

/Users/mwr/Documents/repositories/obi-one/obi_one/scientific/afferent_synapse_finder/find_specified_afferent_synapses.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pre_node_props = pandas.concat(pre_node_props, axis=0, names=["source_population", "edge_population"],


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type


/Users/mwr/Documents/repositories/obi-one/obi_one/scientific/afferent_synapse_finder/find_specified_afferent_synapses.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pre_node_props = pandas.concat(pre_node_props, axis=0, names=["source_population", "edge_population"],


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type
1820,POm,POm__S1nonbarrel_neurons__chemical,102540,166,68,1.033777,4794,1337,NaN,4
1831,POm,POm__S1nonbarrel_neurons__chemical,102551,162,91,0.347443,6022,1337,NaN,4
1819,POm,POm__S1nonbarrel_neurons__chemical,102539,168,21,3.185710,3832,1337,NaN,4


/Users/mwr/Documents/repositories/obi-one/obi_one/scientific/afferent_synapse_finder/find_specified_afferent_synapses.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pre_node_props = pandas.concat(pre_node_props, axis=0, names=["source_population", "edge_population"],


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type


## Case 2: Synapse clusters

There are two options: (1) Return all synapses that are within a specified maximum distance of the center of a cluster. (2) Return the n closest synapses to a cluster center.

Now, an additional column "cluster_id" is added.

In [4]:
# All within the distance
tst = obi.ClusteredSynapsesByMaxDistance(
    n_clusters=3,
    cluster_max_distance=25.0
)
display(tst.synapses_on(circ, node_set, node_id))

# The 25 closest
tst = obi.ClusteredSynapsesByCount(
    n_clusters=3,
    n_per_cluster=25,
    pre_node_populations=("VPM", "POm")  # Additional filters still enabled
)
display(tst.synapses_on(circ, node_set, node_id))


,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
76,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185762,112,61,4.696655,4529,1337,3
109,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185795,112,59,1.138809,6648,1337,3
110,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185796,112,55,0.718337,6648,1337,3
261,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185947,112,61,9.148268,16208,1337,3
345,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186031,112,46,0.731925,19660,1337,3
369,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186055,112,53,0.905540,20340,1337,3
752,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186438,112,61,6.166913,110176,1337,3
774,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186460,112,61,6.493636,111297,1337,3
798,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186484,112,60,4.890615,112286,1337,3
805,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186491,112,60,3.097389,112286,1337,3


,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
2109,0,VPM,VPM__S1nonbarrel_neurons__chemical,178428,122,35,0.196120,6372,1337,3
2351,0,VPM,VPM__S1nonbarrel_neurons__chemical,178670,122,34,0.116039,12047,1337,3
2045,0,VPM,VPM__S1nonbarrel_neurons__chemical,178364,122,38,1.423923,4487,1337,3
2352,0,VPM,VPM__S1nonbarrel_neurons__chemical,178671,122,43,0.054273,12047,1337,3
2227,0,VPM,VPM__S1nonbarrel_neurons__chemical,178546,122,24,0.480743,8615,1337,3
...,...,...,...,...,...,...,...,...,...,...
1977,2,VPM,VPM__S1nonbarrel_neurons__chemical,178296,132,0,0.258751,2958,1337,3
2199,2,VPM,VPM__S1nonbarrel_neurons__chemical,178518,151,12,0.307418,7504,1337,4
2136,2,VPM,VPM__S1nonbarrel_neurons__chemical,178455,132,2,2.312972,7052,1337,3
1988,2,VPM,VPM__S1nonbarrel_neurons__chemical,178307,104,12,0.451906,3187,1337,3


## Case 3: Also consider path distance


In [5]:
# All within the distance
tst = obi.ClusteredPDSynapsesByMaxDistance(
    n_clusters=3,
    cluster_max_distance=25.0,
    soma_pd_mean=100.0,
    soma_pd_sd=10.0
)
display(tst.synapses_on(circ, node_set, node_id))

# The 25 closest
tst = obi.ClusteredPDSynapsesByCount(
    n_clusters=3,
    n_per_cluster=25,
    soma_pd_mean=400.0,
    soma_pd_sd=10.0,
    pre_node_populations=("VPM", "POm")
)
display(tst.synapses_on(circ, node_set, node_id))

,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
122,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185808,93,143,0.348943,7580,1337,3
238,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185924,93,127,0.532427,15486,1337,3
272,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1185958,93,115,0.277223,16627,1337,3
461,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186147,93,104,1.056867,25279,1337,3
519,0,S1nonbarrel_neurons,S1nonbarrel_neurons__S1nonbarrel_neurons__chem...,1186205,93,113,0.378717,28262,1337,3
...,...,...,...,...,...,...,...,...,...,...
2279,2,VPM,VPM__S1nonbarrel_neurons__chemical,178598,175,0,0.277927,9773,1337,4
2318,2,VPM,VPM__S1nonbarrel_neurons__chemical,178637,176,8,0.304326,11184,1337,4
2358,2,VPM,VPM__S1nonbarrel_neurons__chemical,178677,176,8,0.247544,12105,1337,4
2366,2,VPM,VPM__S1nonbarrel_neurons__chemical,178685,176,7,0.797054,12383,1337,4


,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
1837,0,POm,POm__S1nonbarrel_neurons__chemical,102557,158,34,0.593004,7100,1337,4
1814,0,POm,POm__S1nonbarrel_neurons__chemical,102534,159,3,0.429863,3206,1337,4
1826,0,POm,POm__S1nonbarrel_neurons__chemical,102546,160,6,0.125907,5398,1337,4
1823,0,POm,POm__S1nonbarrel_neurons__chemical,102543,160,14,0.368883,5285,1337,4
1824,0,POm,POm__S1nonbarrel_neurons__chemical,102544,159,16,0.845747,5285,1337,4
...,...,...,...,...,...,...,...,...,...,...
2034,2,VPM,VPM__S1nonbarrel_neurons__chemical,178353,134,5,0.073935,4416,1337,3
2276,2,VPM,VPM__S1nonbarrel_neurons__chemical,178595,136,3,5.043669,9773,1337,3
2221,2,VPM,VPM__S1nonbarrel_neurons__chemical,178540,133,8,0.688999,8474,1337,3
2168,2,VPM,VPM__S1nonbarrel_neurons__chemical,178487,133,8,0.417249,7342,1337,3


## Case 4: Consider connections instead of individual synapses

This one may be useful for spike replay. The idea is that instead of selecting individual synapses, the code selects from presynaptic neurons.

This can be applied by setting "merge_multiple_syns_con" to True in any of the Blocks. 

In that case, path distances are considered by averaging them over synapse from the same presynaptic neuron. That is, for each presynaptic neuron, its path distance to all others and to the soma is the mean of its synapses onto the morphology. 
It is up to the user to decide how much sense that makes in their use case. Otherwise, they can stick to the non-path-distance based selectors.

#### Note
The output in that case is much leaner: It contains only the required information to identify the presynaptic neuron selected. This is because for most of the outputs of the dataframe, such as section id, averaging does not make sense. So they are dropped instead.

In [6]:
tst = obi.PathDistanceConstrainedNumberOfSynapses(
    n=25,
    soma_pd_mean=400.0,
    soma_pd_sd=10.0,
    pre_node_populations=("VPM", "POm"),
    merge_multiple_syns_con=True
)
display(tst.synapses_on(circ, node_set, node_id))

,source_population,@source_node
26,VPM,3352
84,VPM,11100
2,POm,2498
55,VPM,7342
68,VPM,8615
81,VPM,10689
16,POm,6985
78,VPM,9773
54,VPM,7307
66,VPM,8419
